In [1]:

!pip install gspread
!pip install oauth2client
!pip install slackclient
!pip install -U snowflake-connector-python
!pip install -U snowflake-snowpark-python
!pip install --upgrade psycopg2
!pip install -U sqlalchemy
!pip install df2gspread

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [14]:
%%capture

from snowflake.snowpark import Session 
import os
import boto3
import base64
import json
from requests import get
from pathlib import Path
from datetime import datetime, timedelta, date
import time
import zipfile
import io
import sqlalchemy
from pathlib import Path
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import pandas as pd
import numpy as np
import psycopg2
import sys
import requests
from botocore.exceptions import ClientError
import importlib
from io import StringIO
import snowflake.connector
from sqlalchemy import create_engine
import asyncio
import nest_asyncio
import re
import pytz
import pandas as pd
import snowflake.connector
import datetime as dt

In [3]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
from datetime import datetime

def imports():
        import os
        import pandas as pd
        import numpy as np
        import psycopg2

import json
import boto3
import os
from pathlib import Path

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])
        
def initialize_env():
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    slack_secret = json.loads(get_secret("prod/slack/reports"))
    rowaa_metabase_access = json.loads(get_secret("prod/metabase/rowaa/user"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["database"]

    os.environ["SLACK_TOKEN"] = slack_secret["token"]

    os.environ["METABASE_USERNAME_ROWAA"] = rowaa_metabase_access["username"]
    os.environ["METABASE_PASSWORD_ROWAA"] = rowaa_metabase_access["password"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"] 

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets
    

def get_from_gsheet(workbook, sheet):
    scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
    initialize_env()
    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
    client = gspread.authorize(creds)
    try:
        wks = client.open(workbook).worksheet(sheet)
        sheet = pd.DataFrame(wks.get_all_records())
    except:
        print(sheet,'failed to fetch data')
    
    return sheet

# Defining Functions

In [4]:
import os
import requests
import json
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns
from io import StringIO
import sys
from datetime import datetime

def ret_metabase(username, password, question):
    base_url = 'https://bi.maxab.info/api'
    base_headers = {'Content-Type': 'application/json'}

    try:
        s_response = requests.post(
            base_url + '/session',
            data=json.dumps({
                'username': username,
                'password': password
            }),
            headers=base_headers)
        s_response.raise_for_status()

        session_token = s_response.json()['id']
        base_headers['X-Metabase-Session'] = session_token

        p_response = requests.post(base_url + '/card/' + str(question) + '/query/csv', headers=base_headers)
        p_response.raise_for_status()

        my_dict = p_response.content
        s = str(my_dict, 'utf-8')
        my_dict = StringIO(s)
        df = pd.read_csv(my_dict)
        return(df)

    except Exception as e:
         print(e)

    
#run query save on metabse with ID, 
#for example here query 1606 is save in my personal collection and I can run it like line below
# ret_metabase(1606)

In [15]:
def assign_data_equal_projects(df, list, assigns):
    df = df.sample(frac=1)  # Shuffle the data
    project_types = df['project_name'].unique()
    
    assigned_data = pd.DataFrame()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.reset_index(drop=True)
        rows_per_agent = len(project_df) // len(list)
        remainder = len(project_df) % len(list)
        
        # Distribute rows equally
        for i, agent in enumerate(list):
            start_idx = i * rows_per_agent
            end_idx = start_idx + rows_per_agent
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            # Handle remainder
            if i < remainder:
                extra_row = project_df.iloc[end_idx:end_idx+1].copy()
                extra_row['agent_assigned'] = agent
                agent_data = pd.concat([agent_data, extra_row])
            
            assigned_data = pd.concat([assigned_data, agent_data])
    
    assigned_data = assigned_data.reset_index(drop=True)
    
    return assigned_data


def clean_column_id(df, column_name):
    # Ensure the column is treated as a string
    df[column_name] = df[column_name].astype(str)
    
    # Replace commas in the string
    df[column_name] = df[column_name].str.replace(',', '')
    
    # Convert back to an integer, if appropriate
    df[column_name] = df[column_name].astype('Int64', errors='ignore')
    
    return df

In [6]:
# assign tasks by mapping 
def assign_data_by_mapping(df, mapping_df):
    # store retail-agent mapping in a dictionary 
    mapping_dict = dict(zip(mapping_df['main_system_id'], mapping_df['agent_assigned']))
    
    
    assigned_agents = []
    # missing_retailers = [] could use it later for viewing errors

    for retailer_id in df['main_system_id']:
        if retailer_id in mapping_dict:
            assigned_agents.append(mapping_dict[retailer_id])
        else:
            assigned_agents.append(None)
            

    
    df['agent_assigned'] = assigned_agents

    df = df.reset_index(drop=True)

    return df

In [7]:
def initialize_env():
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    slack_secret = json.loads(get_secret("prod/slack/reports"))
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["database"]

    os.environ["SLACK_TOKEN"] = slack_secret["token"]

    os.environ["FINTECH_EMONEY_EMAIL"] = fintech_service_account["email_name"]
    os.environ["FINTECH_EMONEY_PASSWORD"] = fintech_service_account["email_password"]

    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))
    os.environ["EGYPT_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["EGYPT_METABASE_PASSWORD"] = metabase_secret["metabase_password"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"] 

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets

In [16]:
def remove_assign(previously_assigned, df, assigns):
    if  previously_assigned.shape[0] != 1:
        previously_assigned[0] = previously_assigned[0].fillna('').astype(str).str.replace(" ","",regex=False)
        previously_assigned = previously_assigned.dropna()
        previously_assigned[0] = previously_assigned[0].astype('float')
        previously_assigned[0] = previously_assigned[0].astype('int')
        df['main_system_id'] = df['main_system_id'].astype('int')
        if df['agent_assigned'].dtype == 'object':
            df['agent_assigned'] = df['agent_assigned'].astype('int')

        # Filter out previously assigned IDs but keep main_system_id == 1
        main_data_to_assign = pd.DataFrame(
            df.loc[~df['main_system_id'].isin(previously_assigned[0].astype(int).values) | (df['main_system_id'] == 1)]
        )    

        main_data_to_assign['main_system_id'] = main_data_to_assign['main_system_id'].astype('int')
        main_data_to_assign = main_data_to_assign.groupby('agent_assigned').head(assigns)
        return main_data_to_assign
    else:
        df['main_system_id'] = df['main_system_id'].astype('int')
        main_data_to_assign = df.groupby('agent_assigned').head(assigns)
        return main_data_to_assign

def assign_data_equal_projects(df, list, assigns):
    df = df.sample(frac=1)  # Shuffle the data
    project_types = df['project_name'].unique()
    
    assigned_data = pd.DataFrame()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.reset_index(drop=True)
        rows_per_agent = len(project_df) // len(list)
        remainder = len(project_df) % len(list)
        
        # Distribute rows equally
        for i, agent in enumerate(list):
            start_idx = i * rows_per_agent
            end_idx = start_idx + rows_per_agent
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            # Handle remainder
            if i < remainder:
                extra_row = project_df.iloc[end_idx:end_idx+1].copy()
                extra_row['agent_assigned'] = agent
                agent_data = pd.concat([agent_data, extra_row])
            
            assigned_data = pd.concat([assigned_data, agent_data])
    
    assigned_data = assigned_data.reset_index(drop=True)
    
    return assigned_data

def check_distribution(df, agent_list):
    """
    Checks if the distribution of project types across agents is even.
    Returns a dictionary with the count of rows per agent for each project type.
    """
    distribution = {}
    project_types = df['project_name'].unique()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        distribution[project] = project_df['agent_assigned'].value_counts().reindex(agent_list, fill_value=0)
    
    return distribution

def redistribute_rows(df, agent_list):
    """
    Redistributes rows among agents if the distribution is uneven.
    """
    project_types = df['project_name'].unique()
    redistributed_data = pd.DataFrame()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.sample(frac=1).reset_index(drop=True)  # Shuffle data
        rows_per_agent = len(project_df) // len(agent_list)
        remainder = len(project_df) % len(agent_list)
        
        start_idx = 0
        
        for i, agent in enumerate(agent_list):
            end_idx = start_idx + rows_per_agent + (1 if i < remainder else 0)
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            redistributed_data = pd.concat([redistributed_data, agent_data])
            
            start_idx = end_idx
    
    redistributed_data = redistributed_data.reset_index(drop=True)
    
    return redistributed_data

def ensure_correct_dispatching(df, agent_list, final_old_assign):
    """
    Ensures the dispatching is done correctly by checking and redistributing rows if necessary.
    `final_old_assign` rows are excluded from redistribution.
    """

    # Exclude final_old_assign from the distribution check, but keep rows with main_system_id == 1
    new_assignments = df[~df['main_system_id'].isin(final_old_assign['main_system_id']) | (df['main_system_id'] == 1)]
    
    distribution = check_distribution(new_assignments, agent_list)
    
    # Check if any project has an uneven distribution across agents
    uneven_distribution = any(distribution[project].nunique() > 1 for project in distribution)
    
    if uneven_distribution:
        print("Uneven distribution detected. Redistributing rows...")
        new_assignments = redistribute_rows(new_assignments, agent_list)
    else:
        print("Distribution is even. No redistribution needed.")
    
    # Combine the redistributed new assignments with the final_old_assign
    final_data = pd.concat([final_old_assign, new_assignments], ignore_index=True)
    
    return final_data

def check_distribution_df(df, agent_list):
    """
    Checks if the distribution of project types across agents is even.
    Returns a DataFrame with the count of rows per agent for each project type.
    """
    project_types = df['project_name'].unique()
    
    distribution_data = []
    current_time = datetime.now()  # Get the current datetime
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        agent_counts = project_df['agent_assigned'].value_counts().reindex(agent_list, fill_value=0)
        
        for agent, count in agent_counts.items():
            distribution_data.append({
                'project_type': project,
                'agent_assigned': agent,
                'count': count,
                'datetime': dt.datetime.now()  + timedelta(hours=3)
            })
    
    distribution_df = pd.DataFrame(distribution_data)
    
    return distribution_df


In [19]:
initialize_env()
now = datetime.now() + timedelta(hours=3)
hour = int(str(now.time())[0:2])

attendance = ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], 13502)
attendance_copy = attendance.copy()

attendance_copy['start_time'] = attendance_copy['start_time'].astype(int)
attendance_copy['end_time'] = attendance_copy['end_time'].astype(int)

attendance_copy['assignment_start_time'] = attendance_copy['start_time'] - 1
attendance_copy['assignment_end_time'] = attendance_copy['end_time'] - 1

attendance_copy['assign_data'] = np.where(
    (hour >= attendance_copy['assignment_start_time']) & (hour <= attendance_copy['assignment_end_time']),
    'yes',
    'no')

task_based_agents = attendance_copy.loc[
    (attendance_copy['project'] == 'task_based') & (attendance_copy['assign_data'] == 'yes')
]

task_based_list = task_based_agents['agent_id'].values.tolist()
number_of_task_based_agents = len(task_based_list)

time.sleep(15)

if number_of_task_based_agents != 0:
    # setup the environment:
    initialize_env()

    scope = [
        "https://spreadsheets.google.com/feeds",
        'https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]

    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
    client = gspread.authorize(creds)

    query_ids = [59874, 36299, 35981, 38188, 49423, 49557, 49556, 39940, 49566, 55124, 55045, 56316, 59170, 59188, 59874 ]

    queries_for_mapped = ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], 59585)
    queries_for_mapped.columns = queries_for_mapped.columns.str.lower()

    dataframes = [ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], qid) for qid in query_ids]
    for df in dataframes:
        df.columns = map(str.lower, df.columns)

    # Concatenate the DataFrames along the rows (union all)
    df_unfiltered = pd.concat(dataframes, ignore_index=True)

    # Excluding blacklisted retailers
    blacklisted_retailers = [214101,663943,667569,523168,604291,525506,671719,266692,26115,373055]

    df_raw = df_unfiltered[~df_unfiltered['main_system_id'].isin(blacklisted_retailers)]
    df_raw = clean_column_id(df_raw, 'main_system_id')
    queries_for_mapped = queries_for_mapped[~queries_for_mapped['main_system_id'].isin(blacklisted_retailers)]

    # Fetching old assigns data

    previous_calls = ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], 35299)
    previous_calls = clean_column_id(previous_calls, 'main_system_id')

    df = df_raw.merge(previous_calls, on='main_system_id', how='left')
    exclude_same_assigns = clean_column_id(previous_calls,'main_system_id')
    task_based = pd.DataFrame(df_raw.loc[~df_raw['main_system_id'].isin(exclude_same_assigns['main_system_id'].astype(int).values)])

    old_assgns = pd.DataFrame(df_raw.loc[df_raw['main_system_id'].isin(exclude_same_assigns['main_system_id'].astype(int).values)])
    old_assgns = clean_column_id(old_assgns, 'main_system_id')
    df_1 = old_assgns.merge(previous_calls, on='main_system_id', how='left')
    final_old_assign = df_1[["main_system_id", "retailer_mobile_number","retailer_name","description","reward","balance","offer","agent_assigned","project_name"]]

    # Define the range of priorities
    priority_range = range(1, 16)  # 1 to 15 inclusive

    # Initialize a list to store the results
    main_data_list = []

    mapping_sheet = client.open('EG Telesales Cycle Assignment')
    mapping_worksheet = mapping_sheet.worksheet('Sheet1')  
    mapping_data = mapping_worksheet.get_all_values()
    mapping_df = pd.DataFrame(mapping_data[1:], columns=mapping_data[0])  # Convert to DataFrame first
    mapping_df = clean_column_id(mapping_df, 'main_system_id')
    mapping_df = clean_column_id(mapping_df, 'agent_assigned')

    mapping_df = mapping_df[mapping_df['agent_assigned'].isin(task_based_list)]
    mapped_data = assign_data_by_mapping(queries_for_mapped, mapping_df) 
    main_data_list.append(mapped_data)

    # Iterate over each priority level
    for priority in priority_range:
        # Filter the DataFrame for the current priority
        priority_df = task_based[task_based["priority"] == priority].reset_index(drop=True)

        # Assign data to projects
        main_data = assign_data_equal_projects(priority_df, task_based_list, 2000)

        # Append the result to the list
        main_data_list.append(main_data)

    # Concatenate all the main_data DataFrames into one
    main_data_total = pd.concat(main_data_list, ignore_index=True)

    # Filter the data to include the required columns
    main_data = main_data_total[
        ["main_system_id", "retailer_mobile_number", "retailer_name", "description", "reward", "balance", "offer", "agent_assigned", "project_name"] ]

    # Separate rows where main_system_id == 1
    main_system_id_1 = main_data[main_data['main_system_id'] == 1]

    # Remove duplicates for rows where main_system_id is NOT 1
    other_main_system_ids = main_data[main_data['main_system_id'] != 1]
    other_main_system_ids = other_main_system_ids.drop_duplicates(subset=['main_system_id'])

    # Combine both parts back together
    main_data = pd.concat([main_system_id_1, other_main_system_ids], ignore_index=True)


    sheet = client.open('[HOURLY] TASK-BASED Data')
    sheet_instance = sheet.worksheet('Data')
    assignments = sheet_instance.get('G5:G')
    assignments_df = pd.DataFrame.from_dict(assignments)

    df = main_data.copy()

    # Remove previous assignments and filter 
    main_data_to_assign = remove_assign(assignments_df, df, 40)
    final_old_assign_new = remove_assign(assignments_df, final_old_assign, 5)

    # Filter the dataframes based on 'agent_assigned' being in task_based_list
    filtered_df = main_data_to_assign[main_data_to_assign['agent_assigned'].isin(task_based_list)]
    filtered_df_old = final_old_assign_new[final_old_assign_new['agent_assigned'].isin(task_based_list)]

    # Importing data into agents' sheet
    final_data_to_assign = filtered_df.drop_duplicates(subset='main_system_id', keep='first').copy()
    final_old_assign_new = filtered_df_old.drop_duplicates(subset='main_system_id', keep='first').copy()

    # Add 'added_at' timestamp
    final_data_to_assign['added_at'] = now
    final_old_assign_new['added_at'] = now

    # Remove 'index' column if it exists
    final_data_to_assign = final_data_to_assign.drop(columns='index', errors='ignore')

    final_data_to_assign = ensure_correct_dispatching(final_data_to_assign, task_based_list, final_old_assign_new)

    final_data_to_assign = final_data_to_assign.astype(str)
    sheet_df = final_data_to_assign[["main_system_id", "retailer_mobile_number","retailer_name","description","reward","balance","offer","agent_assigned","added_at"]]

query_ids = [59703, 36299, 35981, 38188, 49423, 49557, 49556, 39940, 49566, 55124, 55045, 56316, 59170, 59188 ]

queries_for_mapped = ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], 59585)
queries_for_mapped.columns = queries_for_mapped.columns.str.lower()

dataframes = [ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], qid) for qid in query_ids]
for df in dataframes:
    df.columns = map(str.lower, df.columns)

# Concatenate the DataFrames along the rows (union all)
df_unfiltered = pd.concat(dataframes, ignore_index=True)

# Excluding blacklisted retailers
blacklisted_retailers = [214101,663943,667569,523168,604291,525506,671719,266692,26115,373055]

df_raw = df_unfiltered[~df_unfiltered['main_system_id'].isin(blacklisted_retailers)]
df_raw = clean_column_id(df_raw, 'main_system_id')
queries_for_mapped = queries_for_mapped[~queries_for_mapped['main_system_id'].isin(blacklisted_retailers)]

# Fetching old assigns data

previous_calls = ret_metabase(os.environ["EGYPT_METABASE_USERNAME"], os.environ["EGYPT_METABASE_PASSWORD"], 35299)
previous_calls = clean_column_id(previous_calls, 'main_system_id')

df = df_raw.merge(previous_calls, on='main_system_id', how='left')
exclude_same_assigns = clean_column_id(previous_calls,'main_system_id')
task_based = pd.DataFrame(df_raw.loc[~df_raw['main_system_id'].isin(exclude_same_assigns['main_system_id'].astype(int).values)])

old_assgns = pd.DataFrame(df_raw.loc[df_raw['main_system_id'].isin(exclude_same_assigns['main_system_id'].astype(int).values)])
old_assgns = clean_column_id(old_assgns, 'main_system_id')
df_1 = old_assgns.merge(previous_calls, on='main_system_id', how='left')
final_old_assign = df_1[["main_system_id", "retailer_mobile_number","retailer_name","description","reward","balance","offer","agent_assigned","project_name"]]

# Define the range of priorities
priority_range = range(1, 16)  # 1 to 15 inclusive

# Initialize a list to store the results
main_data_list = []

mapping_sheet = client.open('EG Telesales Cycle Assignment')
mapping_worksheet = mapping_sheet.worksheet('Sheet1')  
mapping_data = mapping_worksheet.get_all_values()
mapping_df = pd.DataFrame(mapping_data[1:], columns=mapping_data[0])  # Convert to DataFrame first
mapping_df = clean_column_id(mapping_df, 'main_system_id')
mapping_df = clean_column_id(mapping_df, 'agent_assigned')

mapping_df = mapping_df[mapping_df['agent_assigned'].isin(task_based_list)]
mapped_data = assign_data_by_mapping(queries_for_mapped, mapping_df) 
main_data_list.append(mapped_data)

# Iterate over each priority level
for priority in priority_range:
    # Filter the DataFrame for the current priority
    priority_df = task_based[task_based["priority"] == priority].reset_index(drop=True)

    # Assign data to projects
    main_data = assign_data_equal_projects(priority_df, task_based_list, 2000)

    # Append the result to the list
    main_data_list.append(main_data)

# Concatenate all the main_data DataFrames into one
main_data_total = pd.concat(main_data_list, ignore_index=True)

# Filter the data to include the required columns
main_data = main_data_total[
    ["main_system_id", "retailer_mobile_number", "retailer_name", "description", "reward", "balance", "offer", "agent_assigned", "project_name"] ]

# Separate rows where main_system_id == 1
main_system_id_1 = main_data[main_data['main_system_id'] == 1]

# Remove duplicates for rows where main_system_id is NOT 1
other_main_system_ids = main_data[main_data['main_system_id'] != 1]
other_main_system_ids = other_main_system_ids.drop_duplicates(subset=['main_system_id'])

# Combine both parts back together
main_data = pd.concat([main_system_id_1, other_main_system_ids], ignore_index=True)


sheet = client.open('[HOURLY] TASK-BASED Data')
sheet_instance = sheet.worksheet('Data')
assignments = sheet_instance.get('G5:G')
assignments_df = pd.DataFrame.from_dict(assignments)
# print(assignments_df)
df = main_data.copy()

# Remove previous assignments and filter 
main_data_to_assign = remove_assign(assignments_df, df, 40)
final_old_assign_new = remove_assign(assignments_df, final_old_assign, 5)

# Filter the dataframes based on 'agent_assigned' being in task_based_list
filtered_df = main_data_to_assign[main_data_to_assign['agent_assigned'].isin(task_based_list)]
filtered_df_old = final_old_assign_new[final_old_assign_new['agent_assigned'].isin(task_based_list)]

# Importing data into agents' sheet
final_data_to_assign = filtered_df.drop_duplicates(subset='main_system_id', keep='first').copy()
final_old_assign_new = filtered_df_old.drop_duplicates(subset='main_system_id', keep='first').copy()

# Add 'added_at' timestamp
final_data_to_assign['added_at'] = now
final_old_assign_new['added_at'] = now

# Remove 'index' column if it exists
final_data_to_assign = final_data_to_assign.drop(columns='index', errors='ignore')

final_data_to_assign = ensure_correct_dispatching(final_data_to_assign, task_based_list, final_old_assign_new)

final_data_to_assign = final_data_to_assign.astype(str)
sheet_df = final_data_to_assign[["main_system_id", "retailer_mobile_number","retailer_name","description","reward","balance","offer","agent_assigned","added_at"]]
print(sheet_df)

google_sh = client.open('one_more_test')
sheet = google_sh.worksheet('Sheet1')
sheet.append_rows([sheet_df.columns.values.tolist()] + sheet_df.values.tolist(), value_input_option="USER_ENTERED")

distribution_df = check_distribution_df(final_data_to_assign, task_based_list)
distribution_df = distribution_df.astype(str)

google_sh = client.open('testing_second')
sheet = google_sh.worksheet('Sheet1')
data_to_import = distribution_df.values.tolist()
sheet.append_rows(data_to_import, value_input_option="USER_ENTERED")

/tmp/ipykernel_13779/4143388731.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].astype(str)
/tmp/ipykernel_13779/4143388731.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.replace(',', '')
/tmp/ipykernel_13779/4143388731.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

Uneven distribution detected. Redistributing rows...


/tmp/ipykernel_13779/84675118.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_system_id'] = df['main_system_id'].astype('int')


Uneven distribution detected. Redistributing rows...
    main_system_id                             retailer_mobile_number  \
0           869000  B40B37323944B913E208D8-3B237F7DA755BFCF33A4654...   
1           868486  B40B37313841BE13E60BDE-15054073F3812CD90D38D7F...   
2           869473  B40B35323E41B91BE606DD-59FE7BD74FBB132B70E66A2...   
3           868815  B40B36343C4AB812E20CDD-4FED25BAE91E52A4620E07F...   
4           868963  B40B37373D41B015E10EDE-5AB1DCBB57229C5912C92AB...   
..             ...                                                ...   
129         872258  B40B37313B41B815E20CDD-A835ACEADF7BDA45234E4FD...   
130         873374  B40B36303C44B015E00FD8-FC757FB33D0C659A2C5007D...   
131          79148  B40B37303C4BBC17E60FDF-E33B7EE22882E7BDA26FD01...   
132          70284  B40B37323F41B916EC07DD-452AFF9812B233BDFAC6CFE...   
133         680873  B40B36313C47BF12E40DDF-53FD68EE1D27C25011CDE17...   

                retailer_name  \
0              احمد مصلح شحات   
1   

{'spreadsheetId': '1hOdDpkJrLoUCxkY22H5xWatkziDoVT00btULtJkahNU',
 'tableRange': 'Sheet1!A1:D115',
 'updates': {'spreadsheetId': '1hOdDpkJrLoUCxkY22H5xWatkziDoVT00btULtJkahNU',
  'updatedRange': 'Sheet1!A116:D139',
  'updatedRows': 24,
  'updatedColumns': 4,
  'updatedCells': 96}}